In [1]:
import os 
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
from skimage import io, transform
import skimage
from tensorflow_vgg import vgg16
from tensorflow_vgg import utils
from tensorflow.python.platform import gfile
import base64
from io import BytesIO
import time

In [2]:
path1 = 'test_pics/qc-01.jpg'
##  360  480 

path2 = 'test_pics/qingcai-01.jpg'
path3 = 'test_pics/jmc-02.jpg'

vege_dict = {0:'jmc', 1:'qc', 2:'qingcai'}

In [3]:
def get_image_height_width(img_path):
    """读取图片，分别返回图片的宽和高"""
    img_arr = io.imread(img_path)
    return [len(img_arr) , len(img_arr[1])]

def load_image(path):
    """根据图片路径读取图片，先进行居中切割，再转为float型，后缩放为224"""
    # load image
    img = skimage.io.imread(path)
#     img = img / 255.0
#     assert (0 <= img).all() and (img <= 1.0).all()
    # print "Original Image Shape: ", img.shape
    # we crop image from center
    short_edge = min(img.shape[:2])
    yy = int((img.shape[0] - short_edge) / 2)
    xx = int((img.shape[1] - short_edge) / 2)
    crop_img = img[yy: yy + short_edge, xx: xx + short_edge]
    # resize to 224, 224
    resized_img = skimage.transform.resize(img, (224, 224))
    return resized_img


def get_binstr_by_resized_img(binstr, cache_path='./cache_pic/', ):
    """解码binstr得到图片，并将其本地缓存，再读取该数据, 返回为数组"""
    
    img = Image.open(BytesIO(base64.b64decode(binstr)))
    
    local_time = time.strftime("%Y-%m-%d  %H_%M_%S",time.localtime(time.time()))
    save_path = cache_path + local_time + '.jpg'
    img.save(save_path, 'JPEG')
    
    resized_img = load_image(save_path)
    
    return resized_img

In [ ]:
height_wid = get_image_height_width(path1)
print ('The origin image shape : ',height_wid)


fn = open(path1, 'rb')
picdata =  fn.read()
binstr = base64.b64encode(picdata)
# image = get_image_by_binbyte(binstr, 360, 480)


ss = get_binstr_by_resized_img(binstr)
print (ss)



bstr0 = base64.b64decode(binstr)

arr0 = np.array(bytearray(bstr0))
# print ('length of arr0', len(arr0))
# print (arr0)

##  使用该方法对编码后的
im = Image.open(BytesIO(base64.b64decode(binstr)))
resutl = im.save('accept.jpg', 'JPEG')  ##对照片做本地缓存

# print(im)

height_wid = get_image_height_width('accept.jpg')
print ('The origin image shape : ',height_wid)

# bstr1 = base64.b64encode(np.array2string(arr0.tostring()))
# bstr2 = base64.b64decode(bstr1)
# arr2 = np.array(bytearray(bstr2))

# print ('length of arr2', len(arr2))

# tple = struct.unpack( '4f', my_data )
# my_array = np.array( tple, dtype=np.float32 )


In [ ]:
f =  open(path1, 'rb')
picData = f.read()
#     print(id(picData))

bb64 = base64.b64encode(picData)

# with open('./image_code', 'wb+') as f:
#     f.write(bb64)
#     print ('图片二进制字节码已保存到：', '/image_code')

# with open('./image_code', 'rb') as f1:
#     binstr = f1.read()
#     print ('已读取二进制字节码')
# print (bb64)

    
pic1_ori = base64.b64decode(bb64)

image = np.matrix(bytearray(pic1_ori))
# image = get_image_by_binbyte(pic1_ori, 360 , 480)
np.reshape(image, newshape=(360, 480))

print (image)
print (len(image))



# print (batch1)


f.close()

In [ ]:
batch_list = []

batch1 = load_image(path1)
batch2 = load_image(path2)

batch_list.append(batch1.reshape((1, 224, 224, 3)))
batch_list.append(batch2.reshape((1, 224, 224, 3)))
test_images = np.concatenate(batch_list, 0)

###  读取pb文件，在默认的graph图中做以下操作：
1. 使用vgg16模型对图像进行特征值提取
2. 读取pb文件，恢复图结构(该图结构不包含瓶颈层)
3. 将待测图片的特征值送入第二步恢复的图结构中，预测其类别

In [ ]:
with tf.Graph().as_default() as g:
    with tf.Session() as sess:
        vgg = vgg16.Vgg16()

        ###  定义input
        input_test = tf.placeholder(tf.float32, shape=[None, 224, 224, 3], name='input_test')
        ###  定义 图像处理后的特征值，
        inputs_fea = tf.placeholder(tf.float32, shape=[None , 4096], name='inputs_fea')
        
        with tf.name_scope("content_vgg"):
            vgg.build(input_test)
            feature_codes = sess.run(vgg.relu6, feed_dict={input_test:test_images})
            print (feature_codes)
    #             print ("feature_codes.shape" , feature_codes.shape)
            assert feature_codes.shape == (2, 4096)
        
        
        # 加入一个256维的全连接的层
        fc = tf.contrib.layers.fully_connected(inputs_fea, 256)
        logits = tf.contrib.layers.fully_connected(fc, 3, activation_fn=None)
        predicted = tf.nn.softmax(logits, name='predicted')

        
        ###  创建saver对象时，必须要定义图结构，如上面的fc logits  predicted 
        saver = tf.train.Saver()
        saver.restore(sess, "checkpoints/zsy.ckpt")  # 注意这里只恢复变量值
        ## 需指定要恢复的操作符或张量， 读取到对应的graph对象中
        
#         g = tf.get_default_graph()
#         g.get_tensor_by_name('predicted:0')
        prob_op = g.get_operation_by_name('predicted')
        pred_result = sess.run(predicted, feed_dict={inputs_fea : feature_codes})
        print (pred_result)
        result_list = tf.argmax(pred_result , 1 ).eval()
        print (result_list)
        print (type(result_list))
        for v in result_list:
            print (vege_dict[v])
        

###  下面代码是用pb文件保存已训练模型权重，还有bug

In [ ]:
# with tf.Graph().as_default() as g:
#     with tf.Session() as sess:
        
# #         sess.run(tf.global_variables_initializer())
#         vgg = vgg16.Vgg16()
        
#         ###  定义input
#         input_test = tf.placeholder(tf.float32, shape=[None, 224, 224, 3], name='input_test')
#         ###  定义 图像处理后的特征值，
#         inputs_fea = tf.placeholder(tf.float32, shape=[None , 4096], name='inputs_fea')
        

#         with tf.name_scope("content_vgg"):
#         # 载入VGG16模型
#             vgg.build(input_test)
#             feature_codes = sess.run(vgg.relu6, feed_dict={input_test:test_images})
#             print (feature_codes)
#     #             print ("feature_codes.shape" , feature_codes.shape)
#             assert feature_codes.shape == (2, 4096)

#         print('input_test.graph =   {0}'.format(input_test.graph))

#         ###   读取pb文件，读取对应张量
# #         output_graph_def = tf.GraphDef()
# #         with open('checkpoints/zsy.pb', "rb") as f:
# #             output_graph_def.ParseFromString(f.read())
# #             _ = tf.import_graph_def(output_graph_def, name="")

#         ###   
#         with gfile.FastGFile('checkpoints/zsy.pb', 'rb') as f:
#             graph_def = tf.GraphDef()
#             graph_def.ParseFromString(f.read())

# #             sess.graph.as_default()
# #             predicted = tf.import_graph_def(graph_def, return_elements=['predicted:0'])
#             fc = tf.contrib.layers.fully_connected(inputs_fea, 256)
#             logits = tf.contrib.layers.fully_connected(fc, 3, activation_fn=None)
#             predicted = tf.nn.softmax(logits, name='predicted')

# #             inputs_fea = sess.graph.get_tensor_by_name('inputs_fea:0')
#             predicted = sess.graph.get_operation_by_name('predicted')


# #             sess.run(predicted, feed_dict={feature_test: np.reshape(batch1, [-1, 224, 224, 3])})  feature_codes
#             result = sess.run(predicted, feed_dict={inputs_fea: feature_codes})
#             print (result)
        


###  格式化输出当前日期，时间的两种方法

In [ ]:
# ##  方法一
# import time
# time_int = time.time()
# local_time = time.strftime("%Y-%m-%d %H:%M:%S",time.localtime(time.time()))
# print (type(time.time()), time.time())
# print (type(local_time), local_time)

# ##  方法二
# import datetime
# local_time = datetime.datetime.now()
# print (type(local_time), local_time)

In [ ]:

import requests
url = "http://www.lance.cn/myweb/request"
headers = {'content-type': 'application/json'}
requestData = {"name": "lance", "age": "28"}
ret = requests.post(url, json=requestData, headers=headers)
if ret.status_code == 200:
    text = json.loads(ret.text)



### 客户端访问api 端口服务

In [ ]:
import requests
import base64

URL = 'http://192.168.30.110:5001/recognize_vege'
requestData = {}

test_path = './test_pics/qingcai-03.jpg'

with open(test_path, 'rb') as f:
    picData = f.read()
    bb64 = base64.b64encode(picData)
    requestData['pic_str'] = bb64

#     print (requestData)
result = requests.post(URL, json=requestData, headers=headers)
if result.status_code == 200:
#     text = json.loads(result.text)
    print (result.test)
#     print( test)
    

###  返回top1的概率值

In [ ]:
def print_prob(prob, file_path):
    synset = [l.strip() for l in open(file_path).readlines()]

    # print prob
    pred = np.argsort(prob)[::-1]

    # Get top1 label
    top1 = synset[pred[0]]
    print(("Top1: ", top1, prob[pred[0]]))
    # Get top5 label
    top5 = [(synset[pred[i]], prob[pred[i]]) for i in range(5)]
    print(("Top5: ", top5))
    return top1

In [ ]:
#  测试阶段

with tf.Session() as sess:
    data = []
    #  从mete元文件恢复graph结构
    saver = tf.train.import_meta_graph('checkpoints/zsy.ckpt.meta')
    saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    
    graph = tf.get_default_graph()
    
    chkp.print_tensors_in_checkpoint_file("checkpoints/zsy.ckpt", tensor_name='', all_tensors=True)
#     print( graph.eval())
    
    
    inputs_ = graph.get_tensor_by_name("inputs_:0")
    feed_dict = {x: data}
    
    logits = graph.get_tensor_by_name('logits_eval:0')  # labels_
    
    classification_result = sess.run(logits, feed_dict)
    
    # 打印出预测矩阵
    print (classification_result)
    # 打印出预测矩阵每一行最大值的索引
    print (tf.argmax(classification_result, 1).eval())
    # 根据索引通过字典对应样本类别
    
    output = []
    output = tf.argmax(classification_result, 1).eval()
    
    for i in range(len(output)):
        print ("第", i+1, "张蔬菜预测：" + vege_dict[output[i]])
    
    